In [0]:
FPL_SINGLE = 15000  # config constant (label clearly in demo)
INCOME_MIN = 10000
INCOME_MAX = 35000
INCOME_STEP = 250

In [0]:
plans = spark.table("workspace.bronze.plans")
subsidy_params = spark.table("workspace.bronze.subsidy_params")

In [0]:
params = subsidy_params.first()

MAX_RATE = params.max_contribution_rate
ZERO_FPL = params.zero_contribution_upto_fpl
CLIFF_FPL = params.cliff_fpl

In [0]:
from pyspark.sql.functions import lit
import pandas as pd

income_values = list(range(INCOME_MIN, INCOME_MAX + INCOME_STEP, INCOME_STEP))
income_df = spark.createDataFrame(pd.DataFrame({"income": income_values}))

In [0]:
from pyspark.sql.functions import col, min as spark_min

silver_plans = plans.filter(col("metal_tier") == "Silver")

benchmark = silver_plans.groupBy("county") \
    .agg(spark_min("base_premium").alias("benchmark_silver_premium"))

In [0]:
plans_income = plans.crossJoin(income_df)
plans_income = plans_income.join(benchmark, on="county", how="left")

In [0]:
from pyspark.sql.functions import when

plans_income = plans_income.withColumn(
    "fpl_ratio", col("income") / lit(FPL_SINGLE)
)

plans_income = plans_income.withColumn(
    "contribution_rate",
    when(col("fpl_ratio") <= ZERO_FPL, 0)
    .when(col("fpl_ratio") >= CLIFF_FPL, MAX_RATE)
    .otherwise(
        MAX_RATE * (col("fpl_ratio") - ZERO_FPL) / (CLIFF_FPL - ZERO_FPL)
    )
)

plans_income = plans_income.withColumn(
    "expected_contribution_monthly",
    (col("income") * col("contribution_rate")) / 12
)

plans_income = plans_income.withColumn(
    "subsidy",
    when(
        col("benchmark_silver_premium") > col("expected_contribution_monthly"),
        col("benchmark_silver_premium") - col("expected_contribution_monthly")
    ).otherwise(0)
)

plans_income = plans_income.withColumn(
    "net_premium",
    when(
        col("base_premium") > col("subsidy"),
        col("base_premium") - col("subsidy")
    ).otherwise(0)
)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

window = Window.partitionBy("plan_id").orderBy("income")

plans_income = plans_income.withColumn(
    "prev_net_premium",
    lag("net_premium").over(window)
)

plans_income = plans_income.withColumn(
    "fragility_slope",
    (col("net_premium") - col("prev_net_premium")) / lit(INCOME_STEP)
)

In [0]:
plans_income.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.silver.subsidy_adjusted_premiums")